# **AI Sales Agent**  

This project aims to automate the role of a salesperson by enabling a virtual agent to sell a company's products efficiently. The company assigns a unique name to the agent along with its branding and product details

Equipped with deep product knowledge and contextual awareness, the AI agent interacts with customers professionally and politely, ensuring a seamless sales experience. Powered by the **Groq LLM model "mixtral-8x7b-32768,"** the agent can answer queries, provide recommendations, and handle customer interactions effectively. Once a customer agrees to purchase, the system generates a secure payment link to complete the transaction via Stripe e.t.c

## Install Libraries

In [ ]:
!pip install langchain langchain-community langchain_huggingface langchain-groq

## Import LLM Api

In [ ]:
from google.colab import userdata
key = userdata.get('groq')

## Import the Libraries

In [ ]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import HumanMessage, AIMessage

## Loading the LLM and Embeddings

In [ ]:
#Load the LLM model from groq
os.environ["GROQ_API_KEY"] = key
llm = ChatGroq(
    temperature=0.1,
    model_name="mixtral-8x7b-32768",
    api_key=key,
)

#load the HuugingFace Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Document Loading and Processing

In [ ]:
# Load the csv file where all the product base data stored
file_path = "/content/drive/MyDrive/electronics_products.csv"
loader = CSVLoader(file_path=file_path)
docs = loader.load()
# print(docs)

# Split the Document
text_spliter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_spliter.split_documents(docs)

# Create an in-memory vector store
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()

## Create a Tools

In [ ]:
# Define the retrieval function
def retrieve_query(query: str):
    """Retrieves documents related to the query."""
    docs = retriever.get_relevant_documents(query)
    return docs


# Create a LangChain Tool
tool = Tool(
    name="retriever",
    func=retrieve_query,
    description="useful for retrieving documents related to a query"
)

## Create a System Prompt

In [ ]:
system_prompt = """
You are {agent_name}, the AI Sales Assistant for {company_name} ({company_business}).

Company Profile:
- Company Name: {company_name}
- Business: {company_business}
- Key Features: {key_features}

Conversation Flow:
1. Introduction:
   - Must need an introduction first
   - Greet the prospect warmly and introduce yourself along with the company.
   - Be polite, respectful, and engage in a natural conversation.

2. Qualification:
   - Confirm that you are speaking with the right person regarding our product/service.
   - Ensure that the prospect has the authority to make purchasing decisions.

3. Understanding Needs:
   - Ask clarifying questions to fully understand the prospect’s requirements.
   - Listen attentively and take note of key points.

4. Needs Analysis:
   - Use open-ended questions to uncover the prospect’s pain points and challenges.
   - Explore their current issues and motivations for seeking a solution.

5. Solution Presentation:
   - Present our product/service as the tailored solution to their needs.
   - Highlight the key features and benefits that directly address their challenges.

6. Confirmation:
   - Confirm the prospect’s interest in the recommended solution.
   - Address any objections and provide further information if needed.

7. Payment:
   - If the prospect agrees to purchase, thank them and provide the payment link:
     https://www.example.com/payment  (replace with your actual payment link).

Additional Guidelines:
- Maintain a natural and interactive conversation flow without overwhelming the prospect.
- Keep the conversation engaging, friendly, and professional.
- Always adhere to company policies and uphold a respectful tone.
"""


# Define the company and agent details
company_name = "TechElectronics"
company_business = "Consumer Electronics Retailer"
agent_name = "Alex"
key_features = "Cutting-edge technology, Competitive pricing, Excellent customer service"

# Format the system prompt with the company and agent details
formatted_system_prompt = system_prompt.format(
    agent_name=agent_name,
    company_name=company_name,
    company_business=company_business,
    key_features=key_features
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", formatted_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

## Building the Agent and Executor

In [ ]:
# Create the agent is created wilth llm that retrived the tool and defined prompt
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor to manage handle the input, converstion history and the when use the tool
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Chat Interface Function

In [ ]:
# Initialize chat history
chat_history = []

# Function to interact with the agent
def chat_with_agent(user_input):
    global chat_history

    # Invoke the agent
    response = agent_executor.invoke({
        "input": user_input,
        "chat_history": chat_history
    })

    # Update chat history
    chat_history.extend([
        HumanMessage(content=user_input),
        AIMessage(content=response["output"])
    ])

    return response["output"]

### Example Conversation

In [1]:
# # Example conversation
# print(chat_with_agent("Hi,How you assist me today."))